<a href="https://colab.research.google.com/github/Daz-Riza-Seriog/Tensorflow_ML/blob/main/2-Customise%20your%20Models/4-%20Week%204/2-Custom%20Layers/Custom_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


# Model subclassing and custom training loops

 ## Coding tutorials
 #### [1. Model subclassing](#coding_tutorial_1)
 #### [2. Custom layers](#coding_tutorial_2)
 #### [3. Automatic differentiation](#coding_tutorial_3)
 #### [4. Custom training loops](#coding_tutorial_4)
 #### [5. tf.function decorator](#coding_tutorial_5)

***
<a id="coding_tutorial_2"></a>
## Custom layers

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Softmax

#### Create custom layers

In [ ]:
# Create a custom layer

class MyLayer(Layer):

  def __init__(self,units,input_dim):
    super(MyLayer, self).__init__()
    self.w = self.add_weight(shape=(input_dim,units),
                             initializer="random_normal")
    self.b = self.add_weight(shape=(units,),
                             initializer="zeros")

  def call(self,inputs):
    return tf.matmul(inputs, self.w)+self.b

dense_layer = MyLayer(3,5)
x = tf.ones((1,5))
print(dense_layer(x))
print(dense_layer.weights)

tf.Tensor([[ 0.1355233  -0.08328712 -0.01313725]], shape=(1, 3), dtype=float32)
[<tf.Variable 'Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[ 0.06868129,  0.02244756, -0.00260083],
       [ 0.09474236, -0.02972884,  0.01908007],
       [-0.04554764, -0.06822804, -0.02431686],
       [ 0.00971853, -0.00168329, -0.01028165],
       [ 0.00792875, -0.00609451,  0.00498202]], dtype=float32)>, <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


In [ ]:
print('trainable weights:', len(dense_layer.trainable_weights))
print('non-trainable weights:', len(dense_layer.non_trainable_weights))

trainable weights: 2
non-trainable weights: 0


In [ ]:
# Specify trainable weights

class MyLayer(Layer):

  def __init__(self,units,input_dim):
    super(MyLayer, self).__init__()
    self.w = self.add_weight(shape=(input_dim,units),
                             initializer="random_normal",
                             trainable=False)
    self.b = self.add_weight(shape=(units,),
                             initializer="zeros",
                             trainable=False)

  def call(self,inputs):
    return tf.matmul(inputs, self.w)+self.b

dense_layer = MyLayer(3,5)

In [ ]:
print('trainable weights:', len(dense_layer.trainable_weights))
print('non-trainable weights:', len(dense_layer.non_trainable_weights))

trainable weights: 0
non-trainable weights: 2


In [ ]:
# Create a custom layer to accumulate means of output values

class MyLayerMean(Layer):

  def __init__(self,units,input_dim):
    super(MyLayerMean, self).__init__()
    self.w = self.add_weight(shape=(input_dim,units),
                             initializer="random_normal")
    self.b = self.add_weight(shape=(units,),
                             initializer="zeros")
    self.sum_activation = tf.Variable(initial_value = tf.zeros(units,),
                                      trainable=False)
    self.number_call = tf.Variable(initial_value = 0,
                                      trainable=False)

  def call(self, inputs):
    activations = tf.matmul(inputs, self.w)+self.b
    self.sum_activation.assign_add(tf.reduce_sum(activations,axis=0))
    self.number_call.assign_add(inputs.shape[0])
    return activations, self.sum_activation/tf.cast(self.number_call, tf.float32)

dense_layer = MyLayerMean(3,5)


In [ ]:
# Test the layer

y, activation_means = dense_layer(tf.ones((1, 5)))
print(activation_means.numpy())

y, activation_means = dense_layer(tf.ones((1, 5)))
print(activation_means.numpy())

[-0.01409848  0.0519644  -0.07366792]
[-0.01409848  0.0519644  -0.07366792]


In [ ]:
# Create a Dropout layer as a custom layer

class MyDropout(Layer):

    def __init__(self, rate):
        super(MyDropout, self).__init__()
        self.rate = rate

    def call(self, inputs):
        # Define forward pass for dropout layer
        return tf.nn.dropout(inputs, rate=self.rate)

#### Implement the custom layers into a model

In [ ]:
# Build the model using custom layers with the model subclassing API

class MyModel(Model):

    def __init__(self, units_1, input_dim_1, units_2, units_3):
        super(MyModel, self).__init__()
        # Define layers
        self.layer_1 = MyLayer(units=units_1,input_dim=input_dim_1)
        self.dropout_1 = MyDropout(0.5)
        self.layer_2 = MyLayer(units=units_2,input_dim=units_1)
        self.dropout_2 = MyDropout(0.5)
        self.layer_3 = MyLayer(units=units_3,input_dim=units_2)
        self.softmax = Softmax()

    def call(self, inputs):
        # Define forward pass
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout_1(x)
        x = self.layer_2(x)
        x = tf.nn.relu(x)
        x = self.dropout_2(x)
        x = self.layer_3(x)
        return self.softmax(x)

In [ ]:
# Instantiate a model object

model = MyModel(64,10000,64,46)
print(model(tf.ones((1, 10000))))
model.summary()

tf.Tensor(
[[0.01080257 0.03014451 0.02859122 0.03530771 0.0327859  0.01255666
  0.02226563 0.00428375 0.0062389  0.01769323 0.03507378 0.01688261
  0.00963396 0.03722268 0.01502528 0.04129417 0.01664491 0.02990917
  0.00783163 0.00667897 0.01779986 0.01222302 0.02173064 0.01328472
  0.01314936 0.10842309 0.06511087 0.01010861 0.01572098 0.02114806
  0.01064863 0.03108604 0.01223418 0.0374742  0.01284294 0.01135046
  0.0061338  0.03756585 0.00765466 0.0144057  0.02740783 0.00918023
  0.01526707 0.01662241 0.00850386 0.0260557 ]], shape=(1, 46), dtype=float32)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_layer_2 (MyLayer)        multiple                  640064    
                                                                 
 my_dropout (MyDropout)      multiple                  0         
                                                                 
 my_layer_3 (MyLayer